In [1]:
import tensorflow as tf
import cv2
import imutils

import numpy as np
import matplotlib.pyplot as plt

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [66]:
images = ['plate001.jpg', 'plate002.jpg','plate003.jpg','plate004.jpg','plate005.jpg','plate001.jpg', 'plate002.jpg','plate003.jpg','plate004.jpg']
for i in images:

    
    img = cv2.imread(i)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 13, 15, 15)
    edged = cv2.Canny(gray, 30, 200)
    contours=cv2.findContours(edged.copy(),cv2.RETR_TREE,
                                                cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours,key=cv2.contourArea, reverse = True)[:10]
    screenCnt = 0

    for c in contours:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)
        # if our approximated contour has four points, then
        # we can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
            break
    if isinstance(screenCnt, int) == True:
        print('No NumberPlate Found')
        pass
    else:

        mask = np.zeros(gray.shape,np.uint8)
        new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
        new_image = cv2.bitwise_and(img,img,mask=mask)


        # Now crop
        (x, y) = np.where(mask == 255)
        (topx, topy) = (np.min(x), np.min(y))
        (bottomx, bottomy) = (np.max(x), np.max(y))
        Cropped = img[topx:bottomx+1, topy:bottomy+1]

        plate_image = Cropped
        gray = cv2.cvtColor(Cropped, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (1,1),0)
        binary = cv2.threshold(blur, 180, 255,
                              cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)

        def sort_contours(cnts, reverse = False):
            i = 0
            boundingBoxes = [cv2.boundingRect(c) for c in cnts]
            (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                               key=lambda b: b[1][i], reverse=reverse))
            return cnts

        cont, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        test_roi = Cropped.copy()

        crop_characters = []

        digit_w, digit_h = 30, 60

        for c in sort_contours(cont):
            (x, y, w, h) = cv2.boundingRect(c)
            ratio = h/w
            if 1<=ratio<=3.5:
                if h/plate_image.shape[0]>=0.5: # Select contour which has the height larger than 50% of the plate
                    # Draw bounding box around digit number
                    cv2.rectangle(test_roi, (x, y), (x + w, y + h), (0, 255,0), 2)

                    # Sperate number and give prediction
                    curr_num = thre_mor[y:y+h,x:x+w]
                    curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
                    _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    crop_characters.append(curr_num)

        print("Detect {} letters...".format(len(crop_characters))) 
        cv2.imshow('img', test_roi)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

Detect 7 letters...
Detect 8 letters...
Detect 0 letters...
No NumberPlate Found
No NumberPlate Found
Detect 7 letters...
Detect 8 letters...
Detect 0 letters...
No NumberPlate Found
